In [6]:
import pandas as pd
import numpy as np
import warnings
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import keras

#import glob
warnings.filterwarnings('ignore')
#os.chdir("Dataopen_1109")

In [10]:
data1 = pd.read_csv("dataset_1.csv")
data2 = pd.read_csv("dataset_2.csv")
data3 = pd.read_csv("dataset_3.csv")
data4 = pd.read_csv("dataset_4.csv")
data5 = pd.read_csv("dataset_5.csv")
data6 = pd.read_csv("dataset_6.csv")

In [11]:
data5.columns

Index(['year', 'month', 'ticker', 'list_date', 'company_name', 'icb_industry',
       'ice_supersector', 'country_of_incorporation', 'market_cap', 'eps',
       'p/e', 'p/s'],
      dtype='object')

In [14]:
data5['icb_industry'].unique()

array(['Financial Services', 'Industrial Goods & Services', 'Healthcare',
       'Media', 'Oil & Gas'], dtype=object)

In [26]:
pd.Timestamp(data1.created)

TypeError: Cannot convert input [0    2014-12-10T12:36:33+00:00
1    2017-07-05T00:46:00+00:00
2    2018-05-21T00:42:00+00:00
3    2017-11-02T19:51:00+00:00
4    2014-01-07T22:48:08+00:00
5    2017-12-14T15:26:50+00:00
6    2017-12-23T12:45:00+00:00
7    2019-07-26T07:16:00+00:00
8    2019-04-09T06:13:00+00:00
9    2019-02-07T11:32:00+00:00
Name: created, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp